## PREPROCESSING

### Install and Importing Library/Package

In [1]:
# Installing Library / Packages
%pip install tensorflow==2.17.1


[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: python -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [2]:

%pip install lime


[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: python -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [3]:
%pip install nltk


[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: python -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [4]:
%pip install gensim


[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: python -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [5]:
import tensorflow as tf
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))
print(tf.__version__)

2024-11-24 01:55:58.493635: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-11-24 01:55:58.686646: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-11-24 01:55:58.817066: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-11-24 01:55:58.909079: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-11-24 01:55:58.934274: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-11-24 01:55:59.080704: I tensorflow/core/platform/cpu_feature_gu

Num GPUs Available:  0
2.17.1


In [6]:
import re
import nltk
import gensim
import string
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.express as px
from nltk.corpus import stopwords
from nltk.stem import SnowballStemmer, WordNetLemmatizer
from sklearn.feature_extraction.text import TfidfVectorizer

from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import Sequential
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.layers import Dense, LSTM, Embedding, Bidirectional
from tensorflow.keras.utils import plot_model
from sklearn.metrics import classification_report


#nltk.download("stopwords")
nltk.download("stopwords")
nltk.download('wordnet')
nltk.download('omw-1.4')


stop_words = set(stopwords.words("english"))
lemmatizer= WordNetLemmatizer()


#Lime
from lime import lime_text
from lime.lime_text import LimeTextExplainer
from lime.lime_text import IndexedString,IndexedCharacters
from lime.lime_base import LimeBase
from lime.lime_text import explanation

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/codespace/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /home/codespace/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     /home/codespace/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


### Gathering Data

In [7]:
# Read datasets
data_train = pd.read_csv('../datasets/raw/train.txt', names=['Text', 'Emotion'], sep=';')
data_val = pd.read_csv('../datasets/raw/val.txt', names=['Text', 'Emotion'], sep=';')
data_test = pd.read_csv('../datasets/raw/test.txt', names=['Text', 'Emotion'], sep=';')

In [8]:
data_train = data_train.rename(columns={'Text': 'text', 'Emotion': 'label'})
data_val = data_val.rename(columns={'Text': 'text', 'Emotion': 'label'})
data_test = data_test.rename(columns={'Text': 'text', 'Emotion': 'label'})

In [9]:
#print first 5 rows
data_train.head()

,text,label
0,i didnt feel humiliated,sadness
1,i can go from feeling so hopeless to so damned...,sadness
2,im grabbing a minute to post i feel greedy wrong,anger
3,i am ever feeling nostalgic about the fireplac...,love
4,i am feeling grouchy,anger


In [10]:
#print the shape of the data set
print(data_train.shape)

(16000, 2)


In [11]:
#print first 5 rows
data_val.head()

,text,label
0,im feeling quite sad and sorry for myself but ...,sadness
1,i feel like i am still looking at a blank canv...,sadness
2,i feel like a faithful servant,love
3,i am just feeling cranky and blue,anger
4,i can have for a treat or if i am feeling festive,joy


In [12]:
#print the shape of the data set
print(data_val.shape)

(2000, 2)


In [13]:
#print first 5 rows
data_test.head()

,text,label
0,im feeling rather rotten so im not very ambiti...,sadness
1,im updating my blog because i feel shitty,sadness
2,i never make her separate from me because i do...,sadness
3,i left with my bouquet of red and yellow tulip...,joy
4,i was feeling a little vain when i did this one,sadness


In [14]:
#print the shape of the data set
print(data_test.shape)

(2000, 2)


### Assessing Data

#### Melihat informasi sederhana dari data

In [15]:
# Melihat informasi sederhana dari data_train
data_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16000 entries, 0 to 15999
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   text    16000 non-null  object
 1   label   16000 non-null  object
dtypes: object(2)
memory usage: 250.1+ KB


In [16]:
# Melihat informasi sederhana dari data_val
data_val.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2000 entries, 0 to 1999
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   text    2000 non-null   object
 1   label   2000 non-null   object
dtypes: object(2)
memory usage: 31.4+ KB


In [17]:
# Melihat informasi sederhana dari data_test
data_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2000 entries, 0 to 1999
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   text    2000 non-null   object
 1   label   2000 non-null   object
dtypes: object(2)
memory usage: 31.4+ KB


#### Melihat Missing Value

In [18]:
print("\nJumlah Nan Value pada data_train \n",data_train.isna().sum())
print("\nJumlah Nan Value pada data_val \n",data_val.isna().sum())
print("\nJumlah Nan Value pada data_test \n",data_test.isna().sum())


Jumlah Nan Value pada data_train 
 text     0
label    0
dtype: int64

Jumlah Nan Value pada data_val 
 text     0
label    0
dtype: int64

Jumlah Nan Value pada data_test 
 text     0
label    0
dtype: int64


#### Mengecek duplikat data

In [19]:
print("\nJumlah Duplikat Data pada data_train \n",data_train.duplicated().sum())
print("\nJumlah Duplikat Data pada data_val \n",data_val.duplicated().sum())
print("\nJumlah Duplikat Data pada data_test \n",data_test.duplicated().sum())


Jumlah Duplikat Data pada data_train 
 1

Jumlah Duplikat Data pada data_val 
 0

Jumlah Duplikat Data pada data_test 
 0


In [20]:
print("\nJumlah Duplikat Data pada data_train \n",data_train['text'].duplicated().sum())


Jumlah Duplikat Data pada data_train 
 31


#### Mengecek detail di setiap kolom dataframe

In [21]:
data_train.describe()

,text,label
count,16000,16000
unique,15969,6
top,i feel on the verge of tears from weariness i ...,joy
freq,2,5362


In [22]:
data_val.describe()

,text,label
count,2000,2000
unique,1998,6
top,i feel so tortured by it,joy
freq,2,704


In [23]:
data_test.describe()

,text,label
count,2000,2000
unique,2000,6
top,im feeling rather rotten so im not very ambiti...,joy
freq,1,695


In [24]:
#print the rows which are duplicated (duplicated in the text but with different emotions)
data_train[data_train['text'].duplicated() == True]

,text,label
5067,i feel on the verge of tears from weariness i ...,joy
6133,i still feel a craving for sweet food,love
6563,i tend to stop breathing when i m feeling stre...,anger
7623,i was intensely conscious of how much cash i h...,sadness
7685,im still not sure why reilly feels the need to...,surprise
8246,i am not amazing or great at photography but i...,love
9596,ive also made it with both sugar measurements ...,joy
9687,i had to choose the sleek and smoother feel of...,joy
9769,i often find myself feeling assaulted by a mul...,sadness
9786,i feel im being generous with that statement,joy


In [25]:
#print the rows which are duplicated (duplicated in the text but with different emotions)
data_val[data_val['text'].duplicated() == True]

,text,label
603,i have had several new members tell me how com...,joy
1993,i feel so tortured by it,anger


In [26]:
#print the rows which are duplicated (duplicated in the text but with different emotions)
data_test[data_test['text'].duplicated() == True]

,text,label


### Cleaning Data

#### Melakukan Drop Duplikat Data

In [27]:
print("\nJumlah Duplikat Data pada data_train \n",data_train.duplicated().sum())


Jumlah Duplikat Data pada data_train 
 1


In [28]:
data_train.drop_duplicates(inplace=True)
data_train.reset_index(drop=True, inplace=True)

In [29]:
#removing duplicated text
data_train.drop_duplicates(subset=['text'], inplace=True)
data_train.reset_index(drop=True, inplace=True)

In [30]:
#removing duplicated text
data_val.drop_duplicates(subset=['text'], inplace=True)
data_val.reset_index(drop=True, inplace=True)

### EDA

#### Mengecek distribusi data di setiap label

In [31]:
data_train.label.value_counts() *100/ len(data_train)

label
joy         33.502411
sadness     29.206588
anger       13.494896
fear        12.104703
love         8.134511
surprise     3.556891
Name: count, dtype: float64

In [ ]:
fig = px.histogram(data_train, x="label", title="Distribusi Emosi")
fig.show()


In [ ]:
data_val.label.value_counts() *100/ len(data_val)

In [ ]:
fig = px.histogram(data_val, x="label", title="Distribusi Emosi")
fig.show()


In [ ]:
data_test.label.value_counts() *100/ len(data_test)

In [ ]:
fig = px.histogram(data_test, x="label", title="Distribusi Emosi")
fig.show()


#### Mengecek distribusi stopwords dalam train

In [ ]:
data_train['stop_words'] = data_train['text'].apply(lambda x: len([word for word in x.split() if word in stop_words]))
data_train.stop_words.value_counts()

In [ ]:
fig = px.histogram(data_train, x="stop_words", title="Distribusi Stop Word dalam data train")
fig.show()

In [ ]:
data_val['stop_words'] = data_val['text'].apply(lambda x: len([word for word in x.split() if word in stop_words]))
data_val.stop_words.value_counts()

In [ ]:
fig = px.histogram(data_val, x="stop_words", title="Distribusi Stop Word dalam data validasi")
fig.show()

In [ ]:
data_test['stop_words'] = data_test['text'].apply(lambda x: len([word for word in x.split() if word in stop_words]))
data_test.stop_words.value_counts()

In [ ]:
fig = px.histogram(data_test, x="stop_words", title="Distribusi Stop Word dalam data uji")
fig.show()

### Function will be imported

In [ ]:
def lemmatization(text):
    lemmatizer= WordNetLemmatizer()

    text = text.split()

    text=[lemmatizer.lemmatize(y) for y in text]

    return " " .join(text)



def remove_stop_words(text):

    Text=[i for i in str(text).split() if i not in stop_words]
    return " ".join(Text)



def Removing_numbers(text):
    text=''.join([i for i in text if not i.isdigit()])
    return text



def lower_case(text):

    text = text.split()

    text=[y.lower() for y in text]

    return " " .join(text)



def Removing_punctuations(text):
    ## Remove punctuations
    text = re.sub('[%s]' % re.escape("""!"#$%&'()*+,،-./:;<=>؟?@[\]^_`{|}~"""), ' ', text)
    text = text.replace('؛',"", )

    ## remove extra whitespace
    text = re.sub('\s+', ' ', text)
    text =  " ".join(text.split())
    return text.strip()



def Removing_urls(text):
    url_pattern = re.compile(r'https?://\S+|www\.\S+')
    return url_pattern.sub(r'', text)



def remove_small_sentences(df):
    for i in range(len(df)):
        if len(df.text.iloc[i].split()) < 3:
            df.text.iloc[i] = np.nan



def normalize_text(df):
    df.text=df.text.apply(lambda text : lower_case(text))
    df.text=df.text.apply(lambda text : remove_stop_words(text))
    df.text=df.text.apply(lambda text : Removing_numbers(text))
    df.text=df.text.apply(lambda text : Removing_punctuations(text))
    df.text=df.text.apply(lambda text : Removing_urls(text))
    df.text=df.text.apply(lambda text : lemmatization(text))
    return df

def normalized_sentence(sentence):
    sentence= lower_case(sentence)
    sentence= remove_stop_words(sentence)
    sentence= Removing_numbers(sentence)
    sentence= Removing_punctuations(sentence)
    sentence= Removing_urls(sentence)
    sentence= lemmatization(sentence)
    return sentence


In [ ]:
# !unzip /usr/share/nltk_data/corpora/wordnet.zip -d /usr/share/nltk_data/corpora/

### Membuat Model

#### Mengaplikasikan fungsi normalisasi untuk teks

In [ ]:
data_train = normalize_text(data_train)
data_test = normalize_text(data_test)
data_val = normalize_text(data_val)

In [ ]:
#Splitting the text from the labels
X_train = data_train['text']
y_train = data_train['label']

X_test = data_test['text']
y_test = data_test['label']

X_val = data_val['text']
y_val = data_val['label']

#### Encode Labels

In [ ]:
le = LabelEncoder()
y_train = le.fit_transform(y_train)
y_test = le.transform(y_test)
y_val = le.transform(y_val)

In [ ]:
#Convert the class vector (integers) to binary class matrix
y_train = to_categorical(y_train)
y_test = to_categorical(y_test)
y_val = to_categorical(y_val)

In [ ]:
MAX_LENGTH = 500

# Create a tokenizer
tokenizer = Tokenizer(oov_token='UNK')
# Fit the tokenizer on the training data
tokenizer.fit_on_texts(X_train)
# Convert text to sequences of tokens
X_train_seq = tokenizer.texts_to_sequences(X_train)
X_test_seq = tokenizer.texts_to_sequences(X_test)
X_val_seq = tokenizer.texts_to_sequences(X_val)
# Pad sequences to ensure equal length
X_train_pad = pad_sequences(X_train_seq, maxlen=MAX_LENGTH, truncating='pre')
X_test_pad = pad_sequences(X_test_seq, maxlen=MAX_LENGTH, truncating='pre')
X_val_pad = pad_sequences(X_val_seq, maxlen=MAX_LENGTH, truncating='pre')

In [ ]:
# Train Word2Vec model
sentences = [text.split() for text in X_train]
w2v_model = gensim.models.Word2Vec(sentences, vector_size=200, window=5, min_count=5, workers=4)

# Create embedding matrix
embedding_matrix = np.zeros((len(tokenizer.word_index) + 1, 200))
for word, i in tokenizer.word_index.items():
    if word in w2v_model.wv:
        embedding_matrix[i] = w2v_model.wv[word]

In [ ]:
# Build neural network architecture

adam = Adam(learning_rate=0.005)

# Define vocabSize
vocabSize = len(tokenizer.word_index) + 1  # +1 for the padding token or OOV token

model = Sequential()
model.add(Embedding(vocabSize, 200, weights=[embedding_matrix])) # Use X_train_pad to get shape
model.add(Bidirectional(LSTM(256, dropout=0.2,recurrent_dropout=0.2, return_sequences=True)))
model.add(Bidirectional(LSTM(128, dropout=0.2,recurrent_dropout=0.2)))
model.add(Dense(6, activation='softmax'))

model.compile(loss='categorical_crossentropy', optimizer=adam, metrics=['accuracy'])

model.summary()

In [ ]:
#to stop the training when the loss starts to increase
callback = EarlyStopping(
    monitor="val_loss",
    patience=4,
    restore_best_weights=True,
)

In [ ]:
# Fit model
history = model.fit(X_train_pad,
                    y_train,
                    validation_data=(X_val_pad, y_val),
                    verbose=1,
                    batch_size=256,
                    epochs=10,
                    callbacks=[callback]
                   )

In [ ]:
plot_model(model, show_shapes=True, dpi=80)

In [ ]:
#print the overall loss and accuracy
model.evaluate(X_test_pad, y_test)

In [ ]:
data_test['label'].value_counts()

In [ ]:
predicted = model.predict(X_test_pad)
y_pred = predicted.argmax(axis=-1)

print(classification_report(le.transform(data_test['label']), y_pred))

In [ ]:
# Visualize Loss & Accuracy

acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']

epochs = range(len(acc))

plt.plot(epochs, acc, 'r', label='Training accuracy')
plt.plot(epochs, val_acc, 'b', label='Validation accuracy')
plt.title('Training and validation accuracy')
plt.legend()
plt.figure()

plt.plot(epochs, loss, 'r', label='Training Loss')
plt.plot(epochs, val_loss, 'b', label='Validation Loss')
plt.title('Training and validation loss')
plt.legend()

plt.show()